#### Section 1 and 2
* Import the necessary libraries
* Connect with requests and use bs4 to get the table from wikipedia
* Assign the column names to the dataframe made from the table
* clean the data for linespaces and grouping them by the postal code and borough
* merge the spacial data onto the main dataframe to get the final result


In [22]:
import requests,pandas as pd,shapely
from bs4 import BeautifulSoup

In [29]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


##### Getting the table from wiki with beautifulsoup and requests

In [4]:
api='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
s=requests.session()
response=s.get(api)
response.text
soup=BeautifulSoup(response.text)
frame=soup.find('table')
columns=['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = columns)
df = df.dropna()

##### Iterating throught bs4 tags and making the dataframe

In [5]:
l = []
for tr in frame.findAll('tr'):
    td = tr.findAll('td')
    row = [tr.text for tr in td]
    l.append(row)
df=pd.DataFrame(l, columns=columns)
df = df.replace('\n','', regex=True)

##### Grouping by the required columns and comma separting the rest

In [6]:
df=df[df['Borough']!='Not assigned']
df.loc[df.Neighborhood == 'Not assigned', 'Neighborhood'] =df['Borough']
df=df.groupby(['Postalcode','Borough'])['Neighborhood'].agg(','.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
df.shape

(103, 3)

##### Merging the dataframe with the spacial data to get the final result

In [14]:
df_spacial=pd.read_csv('http://cocl.us/Geospatial_data')
df_spacial.rename(columns={'Postal Code':'Postalcode'},inplace=True)
toronto=pd.merge( df,df_spacial, on='Postalcode')
toronto.head(12)


,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [17]:
BBox = (toronto.Longitude.min(),   toronto.Longitude.max(),    toronto.Latitude.min(), toronto.Latitude.max())

In [27]:
from geopy.geocoders import Nominatim 

#Use geopy library to get the latitude and longitude values of Toronto
address = 'Toronto,ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of ',address,' are {}, {}.'.format(latitude, longitude))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geograpical coordinate of  Toronto,ON  are 43.653963, -79.387207.


In [41]:
from geopy.geocoders import Nominatim 

#Use geopy library to get the latitude and longitude values of Toronto
address = 'Toronto,ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
Latitude = location.latitude
Longitude = location.longitude
print('The geograpical coordinate of ',address,' are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[Latitude, Longitude], zoom_start=10)
import folium
# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='black',
        fill=True,
        fill_color='red',
        fill_opacity=0.4,
        parse_html=False).add_to(map_toronto)  
            
map_toronto

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geograpical coordinate of  Toronto,ON  are 43.653963, -79.387207.
